# Ejercicio de BYTE-PAIR ENCODING
Integrantes:
*   Aguilar Valenzuela Luis Hector
*   Camargo Loaiza Julio Andres
*   Minjares Neriz Victor Manuel

# Importaciones

In [1]:
import re, collections

# Funciones

### get_vocab(filename)
Función que recibe el nombre de el archivo de texto y devuelve un vocabulario de palabras con la frecuencia de cada palabra y un separador en cada palabra

In [9]:
# Funcion para tokenizar el texto por palabras/espacios.

def get_vocab(filename):
    # La funcion defaultdict crea un dictionario vacio. 
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            # La función strip quita los espacios al principio y al final de un string
            # La función split separa las palabras y las devuelve en un array
            words = line.strip().split()
            
            # Recorre cada palabra del arreglo de palabras
            for word in words:
                # Aqui se llena el diccionario. Agregara al diccionario el elemnto : un espacio +
                # la palabra + el simbolo de fin de palabra </w>.
                # NOTA : list() aqui puede ser opciones, tal vez.
                vocab[' '.join(list(word)) + ' </w>'] += 1 
    return vocab

### get_stats(vocab)


In [5]:
def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

### marge_vocab

In [6]:
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

### get_tokens

In [7]:
def get_tokens(vocab):
    tokens = collections.defaultdict(int)
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens[token] += freq
    return tokens

# Codigo principal

In [10]:
# Minicorpus a usar :
# https://drive.google.com/file/d/17h_rLrWL2xg3jD0U1CCseeaAd6t17yc0/view?usp=share_linket 

vocab = get_vocab('miniCorpus.txt')

print('==========')
print('Tokens Before BPE')
tokens = get_tokens(vocab)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))
print('==========')

num_merges = 15
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    tokens = get_tokens(vocab)
    print('Tokens: {}'.format(tokens))
    print('Number of tokens: {}'.format(len(tokens)))
    print('==========')

Tokens Before BPE
Tokens: defaultdict(<class 'int'>, {'Y': 2, 'o': 53, ',': 14, '</w>': 139, 'J': 1, 'u': 18, 'a': 56, 'n': 39, 'G': 1, 'l': 36, 'd': 44, 'e': 90, 'A': 1, 'r': 34, 's': 37, 'c': 22, 'i': 42, 'b': 7, 'C': 3, 'á': 1, 'm': 11, 'R': 1, 'y': 12, 't': 23, 'ñ': 3, 'q': 7, 'j': 1, 'f': 2, 'h': 9, 'v': 10, 'p': 17, 'é': 1, 'E': 1, 'g': 6, 'M': 2, 'S': 1, 'í': 3, ';': 2, '.': 2, 'V': 1})
Number of tokens: 40
Iter: 0
Best pair: ('o', '</w>')
Tokens: defaultdict(<class 'int'>, {'Y': 2, 'o': 29, ',': 14, '</w>': 115, 'J': 1, 'u': 18, 'a': 56, 'n': 39, 'G': 1, 'l': 36, 'o</w>': 24, 'd': 44, 'e': 90, 'A': 1, 'r': 34, 's': 37, 'c': 22, 'i': 42, 'b': 7, 'C': 3, 'á': 1, 'm': 11, 'R': 1, 'y': 12, 't': 23, 'ñ': 3, 'q': 7, 'j': 1, 'f': 2, 'h': 9, 'v': 10, 'p': 17, 'é': 1, 'E': 1, 'g': 6, 'M': 2, 'S': 1, 'í': 3, ';': 2, '.': 2, 'V': 1})
Number of tokens: 41
Iter: 1
Best pair: ('e', '</w>')
Tokens: defaultdict(<class 'int'>, {'Y': 2, 'o': 29, ',': 14, '</w>': 91, 'J': 1, 'u': 18, 'a': 56, 'n'